## Our Goals

Exploring a historical dataset of loan disbursements, filled with borrower financial demographics and specific loan details, our aim is to perform a thorough analysis of credit profile behaviors. This analysis will help us identify trends, pinpoint areas of oversight, and devise potential strategies for improvement in the following areas:

- Increasing customer retention rates
- Reducing the percentage of loan defaults
- Refining marketing strategies

 As we dive into the data exploration phase, we anticipate uncovering additional trends and insights. Should these initial observations prove statistically and practically significant we will elaborate further opportunities for business improvement during the discovery phase.



In [ ]:
# # Create a contingency table of loan status and annual income level
# contingency_table = pd.crosstab(raw_df['annual_inc_lvl'], raw_df['loan_status'])

# # Perform the chi-square test
# chi_result = chi2_contingency(contingency_table)

# print('Chi-square statistic:', chi_result.statistic)
# print('P-value:', chi_result.pvalue)
# contingency_table

In [ ]:
# contig_perc = contingency_table.div(contingency_table.sum(axis=1), axis=0)
# plt.figure(figsize=(10, 8))
# sns.heatmap(contig_perc, annot=True, fmt=".2%", cmap="coolwarm", linewidths=.5)
# plt.title('Proportional Heatmap of Loan Status by Income Level')
# plt.ylabel('Loan Status')
# plt.xlabel('Annual Income Level')
# plt.show()

# contig_perc = contingency_table.div(contingency_table.sum(axis=1), axis=0)
# sns.heatmap(contig_perc, annot=True, fmt=".2%", cmap="coolwarm", linewidths=.5)
# plt.title('Proportional Heatmap of Loan Status by Income Level')
# plt.ylabel('Annual Income Level')
# plt.xlabel('Loan Status')
# plt.show()

# Need to calc cramers v to prove practical significance. 
# Cramer's V = 0.05 might be communicated as "There is a very weak association between the two variables. Despite the statistical significance, the impact of one variable on the other in practical terms is minimal."
# Cramer's V = 0.70 could be communicated as "There is a strong association between the variables, suggesting that changes in one are closely linked to changes in the other, which is important for predictive modeling and strategic decision-making."